# Applied Data Science Capstone Project

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course')

Hello Capstone Project Course


# Segmenting & Clustering Neighborhoods in Toronto

## Part 1 - Webscraping and creating a pandas dataframe of the Neighborhoods

In [2]:
#!pip install pandas
#!pip install requests
!pip install bs4

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [33]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [34]:
# Use the requests library to download the webpage
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data=requests.get(url).text

In [35]:
# Create a Beautiful soup object
soup = BeautifulSoup(html_data,'html5lib')

# Find table
table=soup.find('table')

In [36]:
# Clean the table and pick up all the rows that are not labelled 'not assigned'
contents=[]

for row in table.find_all("td"):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(cell)

In [72]:
# Create the dataframe
df = pd.DataFrame(contents)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [75]:
# Change some of the names
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
# Widen the Neighborhood column
pd.set_option('display.max_colwidth', 180)

df.shape

(103, 3)